In [1]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.9 MB/s eta 0:00:00


In [4]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [5]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url,
                   encoding="ISO- 8859-1")


In [7]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [8]:
app = dash.Dash(__name__)

options = [{'label':'All Sites','value':'ALL'},
            {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
            {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
            {'label':'KSC LC-39A','value':'KSC LC-39A'},
            {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}]

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboards',
            style={'textAlign':'center','color':'black'}),
    html.Div([dcc.Dropdown(
        id='site-dropdown',
        options=[
            {'label':'All Sites','value':'ALL'},
            {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
            {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
            {'label':'KSC LC-39A','value':'KSC LC-39A'},
            {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}
        ],
        value='ALL',
        placeholder='Select a Launch Site here',
        searchable=True
    )]),
  html.Br(),
  html.Div(dcc.Graph(id='success-pie-chart')),
  html.Br(),
  html.Div([dcc.RangeSlider(
      id='payload-slider',
      min=0,max=10000,step=1000,
      value=[0,10000]
      ),
  ]),
  html.Div(dcc.Graph(id='success-scatter')),

])

@app.callback(
    Output(component_id='success-pie-chart',component_property='figure'),
    Input(component_id='site-dropdown',component_property='value')
)

def get_pie_chart (entered_site):
  filtered_df=spacex_df
  if entered_site == 'ALL':
    fig = px.pie(filtered_df,values='class',
                 names='Launch Site',
                 title='Total Success Launches by Site')
    return fig

  else:
      filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
      fig = px.pie(filtered_df, names='class',
                    title=f'Total Success Launches in {entered_site}')
  return fig

@app.callback(
    Output(component_id='success-scatter',component_property ='figure'),
    [Input(component_id='site-dropdown',component_property='value'),
     Input(component_id='payload-slider',component_property='value')]
)

def scatter (entered_site,payload):
  filtered = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]

  if entered_site == 'ALL':
    fig = px.scatter(filtered,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for all sites')
    return fig

  else:
    fig = px.scatter(filtered[filtered['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category', title=f"Success count on Payload mass for {entered_site}")
    return fig


if __name__  == '__main__':
  app.run_server(debug=True)

<IPython.core.display.Javascript object>